# Jaxoplanet core from scratch

Inspired by the [autodidax tutorial](https://jax.readthedocs.io/en/latest/autodidax.html) from the JAX documentation, in this tutorial we work through implementing some of the core `jaxoplanet` functionality from scratch, to demonstrate and discuss the choices made within the depths of the codebase.

## Solving Kepler's equation

One core piece of infrastructure provided by `jaxoplanet` is a function to solve Kepler's equation

$$
M = E - e \sin(E)
$$

for the eccentric anomaly $E$ as a function of the eccentricity $e$ and mean anomaly $M$.
There is a lot of literature dedicated to solving this equation efficiently and robustly, and we won't get into all the details here, but there are a few points we should highlight:

1. The methods that are most commonly used in astrophysics to solve this equation are all iterative, using some sort of root finding scheme. While these methods can work well, they tend to be less computationally efficient than non-iterative approaches. Even more importantly for our purposes, non-iterative methods are better suited to massively parallel compute architectures like GPUs. These non-iterative methods typically have a two step form: (i) make a good initial guess ("starter") for $E$, then (ii) use a high order root finding update to refine this estimate. 

2. In most Python codes, the Kepler solver is offloaded to a compiled library, but we will find that we can get comparable performance just using JAX, and relying on its JIT compilation to accelerate the computation.

With these points in mind, we can implement the solver that is included with `jaxoplanet`.

### Pure-Python (+JAX) solver

The solver that we use in `jaxoplanet` is based on the method from [Markley (1995)](https://ui.adsabs.harvard.edu/abs/1995CeMDA..63..101M/abstract).
It is possible to get better CPU performance using more sophisticated methods (see, for example, [Raposo-Pulido & Peláez 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.467.1702R/abstract), and the follow up [Brandt et al. 2021](https://ui.adsabs.harvard.edu/abs/2021AJ....162..186B/abstract)), but these methods are somewhat harder to implement efficiently using JAX's programming model, and we expect that they would not be so easily extensible to GPU or TPU acceleration.
Regardless, we find that JAX's JIT compilation allows us to achieve nearly state-of-the-art runtime performance, even with this simple method.

First we implement a "starter" function which uses Markley's approximation to estimate $E$ as a function of $M$ and $e$:

In [ ]:
import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)


def kepler_starter(mean_anom, ecc):
    ome = 1 - ecc
    M2 = jnp.square(mean_anom)
    alpha = 3 * jnp.pi / (jnp.pi - 6 / jnp.pi)
    alpha += 1.6 / (jnp.pi - 6 / jnp.pi) * (jnp.pi - mean_anom) / (1 + ecc)
    d = 3 * ome + alpha * ecc
    alphad = alpha * d
    r = (3 * alphad * (d - ome) + M2) * mean_anom
    q = 2 * alphad * ome - M2
    q2 = jnp.square(q)
    w = jnp.square(jnp.cbrt(jnp.abs(r) + jnp.sqrt(q2 * q + r * r)))
    return (2 * r * w / (jnp.square(w) + w * q + q2) + mean_anom) / d

Then we implement a third order Householder update to refine this estimate:

In [ ]:
def kepler_refiner(mean_anom, ecc, ecc_anom):
    ome = 1 - ecc
    sE = ecc_anom - jnp.sin(ecc_anom)
    cE = 1 - jnp.cos(ecc_anom)

    f_0 = ecc * sE + ecc_anom * ome - mean_anom
    f_1 = ecc * cE + ome
    f_2 = ecc * (ecc_anom - sE)
    f_3 = 1 - f_1
    d_3 = -f_0 / (f_1 - 0.5 * f_0 * f_2 / f_1)
    d_4 = -f_0 / (f_1 + 0.5 * d_3 * f_2 + (d_3 * d_3) * f_3 / 6)
    d_42 = d_4 * d_4
    dE = -f_0 / (f_1 + 0.5 * d_4 * f_2 + d_4 * d_4 * f_3 / 6 - d_42 * d_4 * f_2 / 24)

    return ecc_anom + dE

Putting these together, we can construct a solver function which includes some extra bookkeeping to handle the range reduction of the inputs:

In [ ]:
@jax.jit
@jnp.vectorize
def kepler_solver_impl(mean_anom, ecc):
    mean_anom = mean_anom % (2 * jnp.pi)

    # We restrict to the range [0, pi)
    high = mean_anom > jnp.pi
    mean_anom = jnp.where(high, 2 * jnp.pi - mean_anom, mean_anom)

    # Solve
    ecc_anom = kepler_starter(mean_anom, ecc)
    ecc_anom = kepler_refiner(mean_anom, ecc, ecc_anom)

    # Re-wrap back into the full range
    ecc_anom = jnp.where(high, 2 * jnp.pi - ecc_anom, ecc_anom)

    return ecc_anom

And that's it! Now we have a solver for Kepler's equation that we can use from JAX.

Some notes:

1. We've called this function `kepler_solver_impl` rather than `kepler_solver`, for reasons that we will get into shortly.
2. This function was decorated with the [`jax.numpy.vectorize` function](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.vectorize.html) which, while not strictly necessary in this case, is useful because it signals that we have implemented our solver for scalar inputs and we let JAX handle the vectorization to arrays of different shapes. Unlike `numpy.vectorize`, the JAX version incurs no runtime overhead when vectorizing.

To check to make sure that our implementation works, let's make sure that that our method actually solves the equation of interest.
We start by generating a grid of known eccentric anomalies and computing the corresponding array of mean anomalies using Kepler's equation.
Then we make sure that our solver returns the correct values.

In [ ]:
import matplotlib.pyplot as plt

ecc = 0.5
true_ecc_anom = jnp.linspace(0, 2 * jnp.pi, 50_000)[:-1]
mean_anom = true_ecc_anom - ecc * jnp.sin(true_ecc_anom)

calc_acc_anom = kepler_solver_impl(mean_anom, ecc)

plt.plot(true_ecc_anom, jnp.abs(calc_acc_anom - true_ecc_anom), "k")
plt.axhline(0, color="k")
plt.xlabel("eccentric anomaly")
plt.ylabel(r"error from Kepler solver");

We enabled double precision above and we can see here that the results are correct to within $<10^{-15}$ of absolute error, which should do just fine for any practical applications.
This error does increase for large eccentricities, but even then it should always perform better than $\sim 10^{-12}$, and a full analysis is beyond the scope of this tutorial.

It's also worth testing the performance of this implementation.
Since we execute these tutorials on Read the Docs, the specific results here will depend on the actual allocated hardware, but here's how you would benchmark this operation on your system:

In [ ]:
# Using a large number of test values to minimize effects of Python overhead
mean_anom_bench = jnp.linspace(0, 2 * jnp.pi, 500_000)[:-1]
%timeit kepler_solver_impl(mean_anom_bench, ecc).block_until_ready()

On my 2020 M1 MacBook Pro, I get a runtime of `7 ms` per loop, and in each loop we're solving Kepler's equation 500,000 times, so that means that each solve costs about `14 ns`.
For comparison, on the same system, the Kepler solver from [RadVel](https://github.com/California-Planet-Search/radvel/blob/b279a88710b628f70c9de4045f86e2f864674dc5/src/_kepler.pyx#L24) which is implemented in C/Cython takes `35 ms` per loop.

We should note that the point here is not to pick on RedVel, and let's be clear that this is by no means a complete comparison!
Instead, we want to highlight that in some cases JIT-compiled JAX code can get similar performance to that provided by optimized low level code with Python bindings.

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

ax = axes[0]
ax.plot(
    mean_anom, jax.vmap(jax.grad(kepler_solver_impl), in_axes=(0, None))(mean_anom, ecc)
)
ax.set_ylabel("dE / dM")

ax = axes[1]
ax.plot(
    mean_anom,
    jax.vmap(jax.grad(kepler_solver_impl, argnums=1), in_axes=(0, None))(
        mean_anom, ecc
    ),
)
ax.set_xlabel("mean anomaly")
ax.set_ylabel("dE / de");

$$
\mathrm{d}M = \mathrm{d}E (1 - e \cos E) - \mathrm{d}e \sin E
$$

$$
\frac{\partial E}{\partial M} = \frac{1}{1 - e \cos E}
$$

$$
\frac{\partial E}{\partial e} = \frac{\sin E}{1 - e \cos E}
$$

In [ ]:
from jax.interpreters import ad


@jax.custom_jvp
def kepler_solver(mean_anom, ecc):
    return kepler_solver_impl(mean_anom, ecc)


@kepler_solver.defjvp
def kepler_solver_jvp(primals, tangents):
    mean_anom, ecc = primals
    d_mean_anom, d_ecc = tangents

    # Run the solver from above to compute `ecc_anom`
    ecc_anom = kepler_solver(mean_anom, ecc)

    # Propagate the derivatives using the implicit function theorem
    dEdM = 1 / (1 - ecc * jnp.cos(ecc_anom))
    dEde = jnp.sin(ecc_anom) * dEdM
    d_ecc_anom = dEdM * make_zero(d_mean_anom) + dEde * make_zero(d_ecc)

    return ecc_anom, d_ecc_anom


def make_zero(tan):
    # This is a helper function to handle symbolic zeros (i.e. parameters
    # that are not being differentiated)
    if type(tan) is ad.Zero:
        return ad.zeros_like_aval(tan.aval)
    else:
        return tan

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

ax = axes[0]
ax.plot(mean_anom, jax.vmap(jax.grad(kepler_solver), in_axes=(0, None))(mean_anom, ecc))
ax.set_ylabel("dE / dM")

ax = axes[1]
ax.plot(
    mean_anom,
    jax.vmap(jax.grad(kepler_solver, argnums=1), in_axes=(0, None))(mean_anom, ecc),
)
ax.set_xlabel("mean anomaly")
ax.set_ylabel("dE / de");